In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
 
# General libraries
import os
import numpy as np
import pandas as pd 
import random
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
 
# Deep learning libraries
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, LeakyReLU, Activation, Lambda, GlobalAveragePooling2D, DepthwiseConv2D, GlobalMaxPooling2D
from tensorflow.keras.layers import Add, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf
 
# Setting seeds for reproducibility
seed = 232
np.random.seed(seed)
tf.random.set_seed(seed)

### **Getting Data from different folds (5-fold cross validation scheme has been employed)**

In [ ]:
input_path = '/content/drive/My Drive/four_class/Fold 2/' #Change it as necessary, This is the base path for data

In [ ]:
# Hyperparameters
img_dims = 227
epochs = 50
batch_size = 32

## **Pre-processing of images**

In [ ]:
X_train = []
X_test = []
y_train = []
y_test = []

for tt in ['train', 'test']:
  for cond in ['/COVID19/', '/NORMAL/','/Bacterial/','/Viral/']:
    for img_name in os.listdir(input_path+tt+cond):
      img = cv2.imread(input_path+tt+cond+img_name, 0)
      try:
        img = cv2.resize(img, (img_dims, img_dims))
      except:
        print(img_name)
      
      #lbp_img = local_binary_pattern(img, P=8, R=8, method='uniform')/255.0
      img = np.dstack([img, img, img])  #Feinting color image channel
      img = img.astype('float32') / 255.0
      #img = img/255.0
      if tt=='train':
        X_train.append(img)
        
        if cond=='/COVID19/':
          label=0
          y_train.append(label)
        elif cond=='/NORMAL/':
          label=1
          y_train.append(label)
        elif cond=='/Bacterial/':
          label=2
          y_train.append(label)
        else:
          label=3
          y_train.append(label)
      else:
        X_test.append(img)
        
        if cond=='/COVID19/':
          label=0
          y_test.append(label)
        elif cond=='/NORMAL/':
          label=1
          y_test.append(label)
        elif cond=='/Bacterial/':
          label=2
          y_test.append(label)
        else:
          label=3
          y_test.append(label)


X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)


In [ ]:
print(X_train.shape,X_test.shape)

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

print(y_train_cat.shape, y_test_cat.shape)

## **First Model to Use (MobileNet)**

In [ ]:
##########MODEL DESCRIPTION GOES HERE ###############
########### THIS IS MobileNet ########## 
 
num_classes=4

IMAGE_SIZE = [227,227]
inceptionv3 = tf.keras.applications.MobileNet(input_shape = IMAGE_SIZE + [3], weights = 'imagenet',include_top = False,pooling='avg')


for layer in inceptionv3.layers[:]:
    layer.trainable = True

#x = GlobalAveragePooling2D(inceptionv3.output)
#x = Flatten()(inceptionv3.output)
x=Dense(256,activation='relu')(inceptionv3.output)
x=Dense(128,activation='relu')(x)
x=Dropout(.2)(x)
#x=Dense(256,activation='relu')(x)
prediction = Dense(num_classes, activation='softmax')(x)

model1 = Model(inputs=inceptionv3.input,outputs=prediction)
opt=Adam(learning_rate=0.0001)

model1.compile(optimizer=opt, loss='categorical_crossentropy', metrics=["accuracy"])
weight_path = '/content/drive/My Drive/Results/Weight/4 class mobilenet weight/4 class mobilenet fold 2.h5'###### Here weights of 1st viral vs bacterial model will be loaded
model1.load_weights(weight_path)
 
model_first = Model(model1.inputs, model1.layers[-5].output)# Here, the final dense layer is layer -1, the dropout before that is -2, then -3 and -4 and Flatten layer is -5. Take what you need.
X_train_first = model_first.predict(X_train)
X_test_first = model_first.predict(X_test)
 
######################### This is where we will save the predictions ################################
save_path = '/content/drive/My Drive/'##########################
np.save(save_path+'Train First Model.npy', X_train_first)
np.save(save_path+'Test First Model.npy', X_test_first)

## **Second Model to Use (InceptionV3)**

In [ ]:
##########MODEL DESCRIPTION GOES HERE ###############
########### THIS IS Inception ########## 
 

num_classes=4

IMAGE_SIZE = [227,227]
inceptionv3 = tf.keras.applications.InceptionV3(input_shape = IMAGE_SIZE + [3], weights = 'imagenet',include_top = False,pooling='avg')


for layer in inceptionv3.layers[:]:
    layer.trainable = True

#x = GlobalAveragePooling2D(inceptionv3.output)
#x = Flatten()(inceptionv3.output)
x=Dense(256,activation='relu')(inceptionv3.output)
x=Dense(128,activation='relu')(x)
x=Dropout(.2)(x)
#x=Dense(256,activation='relu')(x)
prediction = Dense(num_classes, activation='softmax')(x)

model2 = Model(inputs=inceptionv3.input,outputs=prediction)
opt=Adam(learning_rate=0.0001)

model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=["accuracy"])
weight_path = '/content/drive/My Drive/Results/Weight/4 class inception weight/4 class inception fold 2.h5'###### Here weights of 1st viral vs bacterial model will be loaded
model2.load_weights(weight_path)
 
model_second = Model(model2.inputs, model2.layers[-5].output)# Here, the final dense layer is layer -1, the dropout before that is -2, then -3 and -4 and Flatten layer is -5. Take what you need.
X_train_second = model_second.predict(X_train)
X_test_second = model_second.predict(X_test)
 
######################### This is where we will save the predictions ################################
save_path = '/content/drive/My Drive/'##########################
np.save(save_path+'Train Second Model.npy', X_train_second)
np.save(save_path+'Test Second Model.npy', X_test_second)

## **Third Model to Use(Densenet201)**


In [ ]:
##########MODEL DESCRIPTION GOES HERE ###############
########### THIS IS DenseNet201 ########## 

num_classes=4

IMAGE_SIZE = [227,227]
inceptionv3 = tf.keras.applications.DenseNet201(input_shape = IMAGE_SIZE + [3], weights = 'imagenet',include_top = False,pooling='avg')


for layer in inceptionv3.layers[:]:
    layer.trainable = True

x=Dense(256,activation='relu')(inceptionv3.output)
x=Dense(128,activation='relu')(x)
x=Dropout(.2)(x)
#x=Dense(256,activation='relu')(x)
prediction = Dense(num_classes, activation='softmax')(x)

model3 = Model(inputs=inceptionv3.input,outputs=prediction)
opt=Adam(learning_rate=0.0001)

model3.compile(optimizer=opt, loss='categorical_crossentropy', metrics=["accuracy"])
weight_path = '/content/drive/My Drive/Results/Weight/4 class densenet weight/4 class Densenet fold 2.h5'###### Here weights of 1st viral vs bacterial model will be loaded
model3.load_weights(weight_path)
 
model_third = Model(model3.inputs, model3.layers[-5].output)# Here, the final dense layer is layer -1, the dropout before that is -2, then -3 and -4 and Flatten layer is -5. Take what you need.
X_train_third = model_third.predict(X_train)
X_test_third = model_third.predict(X_test)
 
######################### This is where we will save the predictions ################################
save_path = '/content/drive/My Drive/'##########################
np.save(save_path+'Train Third Model.npy', X_train_third)
np.save(save_path+'Test Third Model.npy', X_test_third)

## **Fourth Model to Use(Xception)**

In [ ]:
##########MODEL DESCRIPTION GOES HERE ###############
########### THIS IS Xception ########## 

num_classes=4

IMAGE_SIZE = [227,227]
inceptionv3 = tf.keras.applications.Xception(input_shape = IMAGE_SIZE + [3], weights = 'imagenet',include_top = False,pooling='avg')


for layer in inceptionv3.layers[:]:
    layer.trainable = True

#x = GlobalAveragePooling2D(inceptionv3.output)
#x = Flatten()(inceptionv3.output)
x=Dense(256,activation='relu')(inceptionv3.output)
x=Dense(128,activation='relu')(x)
x=Dropout(.2)(x)
#x=Dense(256,activation='relu')(x)
prediction = Dense(num_classes, activation='softmax')(x)

model4 = Model(inputs=inceptionv3.input,outputs=prediction)
opt=Adam(learning_rate=0.0001)

model4.compile(optimizer=opt, loss='categorical_crossentropy', metrics=["accuracy"])
weight_path = '/content/drive/My Drive/Results/Weight/4 class xception weight/4 class xception fold 2.h5'###### Here weights of 1st viral vs bacterial model will be loaded
model4.load_weights(weight_path)
 
model_fourth = Model(model4.inputs, model4.layers[-5].output)# Here, the final dense layer is layer -1, the dropout before that is -2, then -3 and -4 and Flatten layer is -5. Take what you need.
X_train_fourth = model_fourth.predict(X_train)
X_test_fourth = model_fourth.predict(X_test)
 
######################### This is where we will save the predictions ################################
save_path = '/content/drive/My Drive/'##########################
np.save(save_path+'Train Fourth Model.npy', X_train_fourth)
np.save(save_path+'Test Fourth Model.npy', X_test_fourth)

## **Fifth Model to Use (DenseNet121)**

In [ ]:
##########MODEL DESCRIPTION GOES HERE ###############
########### THIS IS DenseNet121 ########## 

num_classes=4

IMAGE_SIZE = [227,227]
inceptionv3 = tf.keras.applications.DenseNet121(input_shape = IMAGE_SIZE + [3], weights = 'imagenet',include_top = False,pooling='avg')


for layer in inceptionv3.layers[:]:
    layer.trainable = True

#x = GlobalAveragePooling2D(inceptionv3.output)
#x = Flatten()(inceptionv3.output)
x=Dense(256,activation='relu')(inceptionv3.output)
x=Dense(128,activation='relu')(x)
x=Dropout(.2)(x)
#x=Dense(256,activation='relu')(x)
prediction = Dense(num_classes, activation='softmax')(x)

model5 = Model(inputs=inceptionv3.input,outputs=prediction)
opt=Adam(learning_rate=0.0001)

model5.compile(optimizer=opt, loss='categorical_crossentropy', metrics=["accuracy"])
weight_path = '/content/drive/My Drive/Results/Weight/4 class densenet121 weight/4 class densenet121 fold 2.h5'###### Here weights of 1st viral vs bacterial model will be loaded
model5.load_weights(weight_path)
 
model_fifth = Model(model5.inputs, model5.layers[-5].output)# Here, the final dense layer is layer -1, the dropout before that is -2, then -3 and -4 and Flatten layer is -5. Take what you need.
X_train_fifth = model_fifth.predict(X_train)
X_test_fifth = model_fifth.predict(X_test)
 
######################### This is where we will save the predictions ################################
save_path = '/content/drive/My Drive/'##########################
np.save(save_path+'Train Fifth Model.npy', X_train_fifth)
np.save(save_path+'Test Fifth Model.npy', X_test_fifth)

In [ ]:
X_train1 = np.concatenate((X_train_first, X_train_second,X_train_third,X_train_fourth,X_train_fifth), axis=1)
X_test1 = np.concatenate((X_test_first, X_test_second,X_test_third,X_test_fourth,X_test_fifth), axis=1)

models = Sequential()
models.add(Dense(256, activation='relu', input_shape=(X_train1.shape[1], )))
models.add(Dense(128, activation='relu'))
models.add(Dropout(0.2))
models.add(Dense(4, activation='softmax'))
opt=Adam(learning_rate=0.0001)
models.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
models.summary()

In [ ]:
model_name = '4 class Ensemble Fold2'#########################################################################

weight_save_path = '/content/drive/My Drive/Results/Weight/'

lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=2, verbose=2, mode='max', min_lr=0.00001)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3, mode='min')
checkpoint = ModelCheckpoint(weight_save_path+model_name+'.h5', monitor='val_accuracy', save_best_only=True, save_weights_only=True)

In [ ]:
# Fitting the model 
hist = models.fit(X_train1,y_train_cat,epochs=epochs, batch_size=16, validation_split=0.2, callbacks=[checkpoint, lr_reduce])


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax = ax.ravel()

for i, met in enumerate(['accuracy', 'loss']):
    ax[i].plot(hist.history[met])
    ax[i].plot(hist.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

In [ ]:
from sklearn.metrics import classification_report

y_pred = models.predict(X_test1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(np.unique(y_pred_bool))


report = classification_report(y_test, y_pred_bool, output_dict=True)
print(classification_report(y_test, y_pred_bool))

In [ ]:

from tensorflow.keras.utils import plot_model

############## Make Sure to Change this ####################
plot_save_path = '/content/drive/My Drive/Results/ModelPlot/'
hist_save_path = '/content/drive/My Drive/Results/History/'
result_save_path = '/content/drive/My Drive/Results/Result/'
confusion_matrix_save_path = '/content/drive/My Drive/Results/Confusion Matrix/'


hist_df = pd.DataFrame(hist.history)
hist_csv_file = hist_save_path+model_name+'_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

plot_model(models, plot_save_path+model_name+'.png', show_shapes=True)

result_df = report
result_df = pd.DataFrame(result_df).transpose()

print(result_df)

result_df.to_csv(result_save_path+model_name+'_result.csv')

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from seaborn import heatmap
from matplotlib import pyplot as plt

preds = np.argmax(models.predict(X_test1), axis=1)
acc = accuracy_score(y_test, np.round(preds))*100
cm = confusion_matrix(y_test, np.round(preds))
cm_norm = confusion_matrix(y_test, np.round(preds), normalize='true')
#tn, fp, fn, tp = cm.ravel()

print('CONFUSION MATRIX ------------------')

ax = heatmap(cm, cmap='Blues', linecolor='lightblue',linewidths=.5,annot=True,annot_kws={'size': 12,"weight": "bold"}, xticklabels=['BACTERIAL PNEUMONIA','COVID19', 'NORMAL','VIRAL PNEUMONIA'], yticklabels=['BACTERIAL PNEUMONIA','COVID19', 'NORMAL','VIRAL PNEUMONIA'], square=True, fmt='d')

ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 12)
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 12,rotation=0)

plt.savefig(confusion_matrix_save_path+model_name+'.png',dpi=500)
plt.show()

ax = heatmap(cm_norm, cmap='Blues', annot=True,linecolor='lightblue',linewidths=.5,annot_kws={'size': 12,"weight": "bold"}, xticklabels=['BACTERIAL PNEUMONIA','COVID19', 'NORMAL','VIRAL PNEUMONIA'], yticklabels=['BACTERIAL PNEUMONIA','COVID19', 'NORMAL','VIRAL PNEUMONIA'], square=True, fmt='.2f')

ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 12)
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 12,rotation=0)

plt.savefig(confusion_matrix_save_path+model_name+'_normalized.png',dpi=500)